In [ ]:
import torch
# torch.zeros(1).cuda()
torch.cuda.is_available()

In [ ]:
from dataset import get_train_val_locs

train_locs, val_locs = get_train_val_locs(locs_scenario="iiasa_evo_v1", n_val_locs=10000)

In [ ]:
from dataset import load_evo_dataset
evo_ds = load_evo_dataset()
v3_locs = evo_ds.locs

In [ ]:
evo_ds.locs["quarter_id"].astype("string[pyarrow]")

In [ ]:
evo_ds.sample_ids

In [ ]:
v3_locs.columns

In [ ]:
from evotrain.v2 import EvoTrainV2Dataset

In [ ]:
from dataset import EvoNetV1Dataset

In [ ]:
def slash_tile(tile: str):
    if len(tile) != 5:
        raise ValueError(f"tile should be a str of len 5, not {tile}")

    return f"{tile[:2]}/{tile[2]}/{tile[3:]}"

In [ ]:
# evo_ds.locs.lat.abs()
# evo_ds.locs.sort_values("lat").quarter_ids.values
from pathlib import Path
gamma0_path = Path('/projects/TAP/vegteam/training_data/GAMMA0-RAW/v001')
import xarray as xr
location_id = '39PWN_002_25' #'34TCR_000_58'
# row = locs[locs['location_id'] == location_id].iloc[0]
tile = '39PWN'
year = 2020

pth = gamma0_path / 'patches' / slash_tile(tile) / str(year) / f'LCFM_GAMMA0-RAW_{year}_{location_id}_V001.nc'
darr = xr.open_dataset(pth)
# darr

In [ ]:
ds = EvoNetV1Dataset(bands=[
                "B02",
                "B03",
                "B04",
                "B08",
                "B05",
                "B06",
                "B07",
                "B8A",
                "B11",
                "B12",
                "cop-DEM-alt",
                "VV",
                "VH",
                "VHVVRATIO"],
                    locs_scenario='iiasa_evo_v1')

In [ ]:
bands=[
                "VV",
                "VH",
                "VHVVRATIO"]
print(len(bands))
ds.read_feats('29PRM_080_57_20200929', bands)

In [ ]:
ds.read_s1feats('47SNV_008_43_20201101', ["VV",
                "VH",
                "VHVVRATIO"])

In [ ]:
list(set(ds.evo_dataset.locs['iiasa_evo_v1']))

In [ ]:
ds.evo_dataset.locs['quarter_id']

In [ ]:
quarter_id = ds.ids[1]
quarter_id

In [ ]:
df_locs = ds.evo_dataset.locs
row = df_locs[df_locs['quarter_id'] == quarter_id].sample(n=1).iloc[0]

In [ ]:
ds[1]

In [ ]:
list(ds.ids).index('39PWN_002_25_2020Q4')

In [ ]:
list(ds.ids).index('55GEN_046_09_20200105')

In [ ]:
feats, feats_head, probs, weights = ds[12820]

In [ ]:
feats.shape, feats_head.shape

In [ ]:

print(probs[0].shape, weights[0].shape)
print(probs[1].shape, weights[1].shape)
print(probs[2].shape, weights[2].shape)

In [ ]:
ind = 2
probs[ind].shape, weights[ind].shape

In [ ]:
import json

with open("/projects/TAP/vegteam/models_wdk/lsc10/v15/lsc10-v15_test/config.json", "r") as f:
    config = json.load(f)

In [ ]:
from model import Net

In [ ]:
net = Net(**config["model"])

In [ ]:
import numpy as np
x, x_head, y, y_weights = ds[121318]
x = torch.from_numpy(x.numpy()[np.newaxis, ...])
x_head = torch.from_numpy(x_head.numpy()[np.newaxis, ...])
y_pred = net.evonet(x, x_head)

In [ ]:
y_pred = [net._slice_cover(y_pred), net._slice_occlusion(y_pred), net._slice_ecosystems(y_pred)]

In [ ]:
y = [torch.from_numpy(it.numpy()[np.newaxis, ...]) for it in y]

In [ ]:
y_weights = [torch.from_numpy(it.numpy()[np.newaxis, ...]) for it in y_weights]

In [ ]:
print(y_pred[0].shape)
print(y[0].shape)
print(y_weights[0].shape)

In [ ]:
net.on_validation_start()
net.num_classes_components['occlusion']
net._iou_accumulators['occlusion']

In [ ]:
from torch import nn
softmax = nn.Softmax(dim=1)
y_pred_cover = softmax(y_pred[0])
y_pred_occlusion = softmax(y_pred[1])
y_pred_ecosystems = softmax(y_pred[2])
y_pred_softmax = [y_pred_cover, y_pred_occlusion, y_pred_ecosystems]
net._update_iou_accumulators(y_pred_softmax, y, y_weights)

In [ ]:
labels_type = 'occlusion'
net._iou_accumulators[labels_type]

In [ ]:
mean_iou, classes_iou = net._get_iou_metric(labels_type)
classes_dict = net.classes_dict[labels_type]
classes_ids = list(classes_dict.keys())
classes_ids

In [ ]:
def get_accuracy_metrics(labels_type):
    
    

In [ ]:
labels_type = 'occlusion'
tp_sum = net._iou_accumulators[labels_type]['true_positives'].sum()
fp_sum = net._iou_accumulators[labels_type]['false_positives'].sum()

overall_acc = tp_sum / (tp_sum + fp_sum)

In [ ]:
overall_acc

In [ ]:
weighted_iou_sum = 0.0
weight_sum = 0.0

# raise ValueError("Not implemented")  # fix range below
for lab_i in range(self.num_classes_components[labels_type]):
    tp = self._iou_accumulators[labels_type]["true_positives"][lab_i]
    fp = self._iou_accumulators[labels_type]["false_positives"][lab_i]
    fn = self._iou_accumulators[labels_type]["false_negatives"][lab_i]
    sum_tp_fn = tp + fn
    if sum_tp_fn > 0:
        recall = tp / union
        weight = self.classes_weights_tensor_component[labels_type][0, lab_i, 0, 0]
        weighted_recall_sum += recall * weight
        weight_sum += weight
        classes_recall.append(recall.item())
    else:
        # from loguru import logger
        # logger.info(f'{labels_type} {lab_i} {tp}')
        classes_recall.append(float("nan"))

mean_recall = (
    (weighted_iou_sum / weight_sum).item()
    if weight_sum > 0
    else float("nan")
)

In [ ]:
classes_iou

In [ ]:
classes_dict = net.classes_dict[labels_type]

In [ ]:
prob_cover.shape, prob_occlusion.shape, prob_eco.shape, weight_cover.shape, weight_occlusion.shape, weight_eco.shape

In [ ]:
len(v3_locs)

In [ ]:
len(list(set(v3_locs['location_id'])))

In [ ]:
v3_locs.columns

In [ ]:
list(set(v3_locs['selection_1']))

In [ ]:
train_locs[0]

In [ ]:
len(train_locs)

In [ ]:
len(val_locs)

In [ ]:
import pandas as pd
'/projects/TAP/vegteam/training_data/evotrain_v2/metadata/meteo_biome_features_v1.parquet'
tst_meteo = pd.read_parquet('/home/vito/dekeersw/repos/evotrain/src/evotrain/v2/metadata/meteo_biome_features_v1.parquet')

In [ ]:
tst_meteo


In [ ]:
# feats_head
# feats

In [ ]:
# config["model"]

In [ ]:
# y_pred

In [ ]:

labels_type = 'occlusion'
range(net.num_classes_components[labels_type])

In [ ]:
net._iou_accumulators[labels_type]

In [ ]:
net._labels_types
label_type = 'occlusion'
mean_iou, classes_iou = net._get_iou_metric(label_type)

In [ ]:
feats.unsqueeze(0).shape

In [ ]:
y = net(feats.unsqueeze(0), feats_head.unsqueeze(0))

In [ ]:
y = net(feats.unsqueeze(0), feats_head.unsqueeze(0))

In [ ]:
len(y)

In [ ]:
y[0].shape

In [ ]:
y[1].shape

In [ ]:
y[2].shape

In [ ]:
y[0].shape

In [ ]:
weights[0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from labels import binarize_probs
from labels import label_to_rgb
from labels import LSC_COVER_LABELS
nodata_mask = (weights[0][0] < 0.1).squeeze()
lsc_probs = probs[0]
# lsc_probs[:, nodata_mask] = np.nan
lsc_map = binarize_probs(lsc_probs, LSC_COVER_LABELS) 
lsc_map[nodata_mask] = 0
rgb = label_to_rgb(lsc_map, 'cover')

print(rgb.shape)
plt.imshow(np.moveaxis(rgb,[0,1,2],[2,0,1]))

In [ ]:
import numpy as np

In [ ]:
prob = np.ones((12,6,128, 128))
prob_sum = prob.sum(axis=1) 
nbands = prob.shape[1]
prob_sum = np.repeat(np.expand_dims(prob_sum, axis=1), nbands, axis=1)

In [ ]:
prob_sum.shape

In [ ]:
prob[prob_sum>1] = prob[prob_sum>1] / prob_sum[prob_sum>1]

In [ ]:
np.expand_dims((1-prob_sum), axis=1).shape

In [ ]:
prob_sum = prob.sum(axis=1)
prob = np.hstack([prob, np.expand_dims((1-prob_sum), axis=1)])


In [ ]:
prob.shape

In [ ]:
import numpy as np
import xarray as xr
prob_cover = ds.evo_dataset.read_annotation(
            sample_id, annotation_name="lsc-evo-v1-prob-cover-v1"
        )
print(len(prob_cover.band))
sample_id = '55GEN_046_09_20200105'
tst = ds.evo_dataset.read_annotation(
            sample_id, annotation_name="lsc-evo-v1-weight-cover-v1"
        )
xr.concat([tst for it in np.arange(len(prob_cover.band))], dim='band')
# xr.DataArray(np.repeat(tst.data, 4, axis=0), dims=['band', 'y', 'x'],
#             coords=)


In [ ]:
from pathlib import Path
import pandas as pd
_DATASET_PATH_HPC = Path("/projects/TAP/vegteam/training_data/evotrain_lsc/evotrain_lsc_v4_iiasa")
meta = pd.read_parquet(_DATASET_PATH_HPC / 'metadata' / "locs_fracs.parquet")

In [ ]:
len(list(set(meta['location_id'])))